In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from itertools import combinations
import re
import requests
import os

from sklearn.model_selection import TimeSeriesSplit, KFold, GridSearchCV, cross_val_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler, PowerTransformer, RobustScaler, FunctionTransformer
from sklearn.linear_model import LassoCV, Ridge, Lasso, ElasticNetCV
from sklearn.feature_selection import f_regression, RFE, SelectKBest, SequentialFeatureSelector, RFECV, VarianceThreshold, mutual_info_regression, SelectFromModel
from sklearn.inspection import permutation_importance
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

from xgboost import XGBRegressor
from lightgbm import LGBMRegressor

import datetime
import json

In [2]:
df_train = pd.read_csv(os.path.join("..", "data", "dataset_preprocesamiento_train.csv"))
df_test = pd.read_csv(os.path.join("..", "data", "dataset_preprocesamiento_test.csv"))

## Fase 2

### Creación de variables

In [3]:
print(f"Train inicial: {df_train.shape} | Test inicial: {df_test.shape}")

ycol = 'Frio'

num_train = df_train.select_dtypes(include=[np.number]).copy()
candidatas_train = [c for c in num_train.columns if c != ycol and c != 'y' and num_train[c].notna().any()]

top5_features_aprendidas = (
    num_train[candidatas_train].corrwith(num_train[ycol])
    .abs()
    .sort_values(ascending=False)
    .head(5)
    .index.tolist())

print(f"Top 5 features aprendidas: {top5_features_aprendidas}")


split_date = df_test['dia'].min() # Guardamos la fecha de corte
df_full = pd.concat([df_train, df_test]).sort_values(by='dia').reset_index(drop=True)

print(f"DataFrame unificado para procesar: {df_full.shape}")

Train inicial: (913, 118) | Test inicial: (297, 118)
Top 5 features aprendidas: ['Sala Maq (Kw)__Consolidado EE', 'Servicios (Kw)__Consolidado EE', 'KW Gral Planta__Consolidado EE', 'Planta (Kw)__Consolidado EE', 'Agua Planta (Hl)__Consolidado Agua']
DataFrame unificado para procesar: (1210, 118)


#### Lags y Rolling

In [4]:
# --- Lags y Rolling (Tus cambios) ---
ycol = 'Frio'
df_full[f'{ycol}_lag7'] = df_full[ycol].shift(7)

y_obs = df_full[ycol].shift(1) # Frio de ayer
df_full[f'{ycol}_ma3'] = y_obs.rolling(window=3, min_periods=1).mean()
df_full[f'{ycol}_ma7'] = y_obs.rolling(window=7, min_periods=1).mean()

#### Fechas y findes

In [5]:
# --- Fechas Cíclicas y Finde ---
df_full['fecha'] = pd.to_datetime(df_full['dia'], format='%Y-%m-%d', errors='coerce')
dow = df_full['fecha'].dt.dayofweek
df_full['dow_sin'] = np.sin(2*np.pi * dow / 7)
df_full['dow_cos'] = np.cos(2*np.pi * dow / 7)
m  = df_full['fecha'].dt.month
m0 = (m - 1)
df_full['mes_sin'] = np.sin(2*np.pi * m0 / 12)
df_full['mes_cos'] = np.cos(2*np.pi * m0 / 12)
df_full['fin_de_semana'] = df_full['fecha'].dt.dayofweek.isin([5, 6]).astype(int)

#### Ratios e interacciones

In [6]:
# --- Ratios e Interacciones (usando la 'top5_features_aprendidas') ---
top5 = top5_features_aprendidas # Usamos la variable segura
for a, b in combinations(top5, 2):
    df_full[f'{a}x{b}'] = df_full[a] * df_full[b]
for cyc in ['dow_sin', 'dow_cos', 'mes_sin', 'mes_cos']:
    if cyc in df_full.columns:
        for c in top5:
            df_full[f'{c}_x_{cyc}'] = df_full[c] * df_full[cyc]
if 'fin_de_semana' in df_full.columns:
    for c in top5:
        df_full[f'{c}_x_finde'] = df_full[c] * df_full['fin_de_semana']

# --- Ratios de Áreas ---
AREAS = {
    "Elaboración": r"Elab|Elabor|Coci|Cocina|Mosto|Lauter|Macer|Paste",
    "Envasado":    r"Envas|Llen|Linea|L[2345]\b",
    "Bodega":      r"Bodega|Bodeg",
    "Servicios":   r"Servicios|Vapor|Gas|Agua|Aire|Caldera|Compres|Chiller|Sala",
    "Sala_Maq":    r"Sala.*Maq",
}
def safe_div(a, b):
    # Usamos np.where para evitar dividir por cero
    return np.where(b != 0, a / b, 0)

num_cols = df_full.select_dtypes(include=[np.number]).columns
area_cols = {}
for area, pat in AREAS.items():
    regex = re.compile(pat, flags=re.IGNORECASE)
    cols = [c for c in num_cols if regex.search(c)]
    area_cols[area] = cols

area_sum = {}
for area, cols in area_cols.items():
    if cols:
        df_full[f'{area}_sum'] = df_full[cols].sum(axis=1, skipna=True)
        area_sum[area] = df_full[f'{area}_sum']
    else:
        df_full[f'{area}_sum'] = 0.0
        area_sum[area] = df_full[f'{area}_sum']

areas_presentes = list(area_sum.keys())
total_sel = sum(area_sum[a] for a in areas_presentes)
df_full['Consumo_Total_Areas'] = total_sel

for a in areas_presentes:
    df_full[f'{a}_share'] = safe_div(df_full[f'{a}_sum'], df_full['Consumo_Total_Areas'])
for a, b in combinations(areas_presentes, 2):
    df_full[f'ratio_{a}_sobre_{b}'] = safe_div(df_full[f'{a}_sum'], df_full[f'{b}_sum'])
    df_full[f'ratio_{b}_sobre_{a}'] = safe_div(df_full[f'{b}_sum'], df_full[f'{a}_sum'])

df_full = df_full.replace([np.inf, -np.inf], np.nan)

#### Estacionalidad y clima

In [7]:
# --- Estacionalidad y Clima ---
m = df_full['fecha'].dt.month
est_idx = np.select(
    [m.isin([12, 1, 2]), m.isin([3, 4, 5]), m.isin([6, 7, 8]), m.isin([9, 10, 11])],
    [0, 1, 2, 3], default=np.nan
)
df_full['estacion_sin'] = np.sin(2*np.pi * est_idx / 4)
df_full['estacion_cos'] = np.cos(2*np.pi * est_idx / 4)

start_date = df_full["fecha"].min().date().isoformat()
end_date   = df_full["fecha"].max().date().isoformat()
print(f"Buscando clima de {start_date} a {end_date}...")
lat, lon = 32.56717, -116.62509
url = (
    "https://archive-api.open-meteo.com/v1/archive"
    f"?latitude={lat}&longitude={lon}"
    f"&start_date={start_date}&end_date={end_date}"
    "&daily=temperature_2m_mean"
    "&timezone=auto"
)
data = requests.get(url, timeout=60).json()
wx_d = pd.DataFrame({
    "fecha": pd.to_datetime(data["daily"]["time"]),
    "t2m_mean_C": data["daily"]["temperature_2m_mean"],
})
df_full = df_full.merge(wx_d, on="fecha", how="left")

Buscando clima de 2020-07-01 a 2023-10-25...


#### Limpiar y separar

In [8]:
df_processed = df_full.iloc[7:].copy()
print(f"Filas eliminadas por NaNs de lags: 7. Quedan {df_processed.shape[0]} filas.")

# 3b. Rellenar cualquier otro NaN que quede (del merge de clima, ratios, etc.)
df_processed = df_processed.fillna(0) # O puedes usar un imputador aquí si prefieres

# 3c. Volver a separar
df_train_final = df_processed[df_processed['dia'] < split_date].copy()
df_test_final = df_processed[df_processed['dia'] >= split_date].copy()

Filas eliminadas por NaNs de lags: 7. Quedan 1203 filas.


### Selección de variables

#### Separamos en x, y train y x, y test

In [9]:
ycol = 'y'

# --- Creamos X_train, y_train, X_test, y_test ---
# (Esta vez, X_train y X_test se quedan con las columnas de texto por ahora)
X_train_full = df_train_final.drop(columns=[ycol], errors='ignore')
y_train = df_train_final[ycol]

X_test_full = df_test_final.drop(columns=[ycol], errors='ignore')
y_test = df_test_final[ycol]

# 1. Creamos una lista de features que SÍ son numéricas
numeric_features = X_train_full.select_dtypes(include=np.number).columns.tolist()

# 2. Creamos los DataFrames X_train y X_test solo con esas columnas
X_train = X_train_full[numeric_features].copy()
X_test = X_test_full[numeric_features].copy()
X_test = X_test[X_train.columns]

print(f"Listos para seleccionar. {len(numeric_features)} features numéricas iniciales.")

Listos para seleccionar. 192 features numéricas iniciales.


#### Métodos de varianza, F-Score y Mutual Info 

In [10]:
# --- A. Filtro por Varianza (VarianceThreshold) ---
scaler_var = MinMaxScaler()
scaler_var.fit(X_train) 

X_train_scaled_var = pd.DataFrame(scaler_var.transform(X_train), columns=X_train.columns)
X_test_scaled_var = pd.DataFrame(scaler_var.transform(X_test), columns=X_test.columns)

var_selector = SelectKBest(score_func=lambda X, y: X.var(axis=0), k=30)
var_selector.fit(X_train_scaled_var, y_train) # 'y_train' no se usa, pero la API lo pide

features_variance = X_train.columns[var_selector.get_support()]
print(f"\n--- 1. Método Varianza (k=30) --- \n{features_variance.tolist()}")


# --- B. Filtro por F-Score (f_regression) ---
X_train_scaled_var = X_train_scaled_var.fillna(0)
X_test_scaled_var = X_test_scaled_var.fillna(0)

f_selector = SelectKBest(score_func=f_regression, k=30)
f_selector.fit(X_train_scaled_var, y_train)

features_fscore = X_train.columns[f_selector.get_support()]
print(f"\n--- 2. Método F-Score (k=30) --- \n{features_fscore.tolist()}")


# --- C. Filtro por Información Mutua (mutual_info_regression) ---
mi_selector = SelectKBest(score_func=mutual_info_regression, k=30)
mi_selector.fit(X_train_scaled_var, y_train)

features_mi = X_train.columns[mi_selector.get_support()]
print(f"\n--- 3. Método Mutual Info (k=30) --- \n{features_mi.tolist()}")


--- 1. Método Varianza (k=30) --- 
['Unnamed: 0', 'Linea 3 (Kw)__Consolidado EE', 'Linea 2 (Kw)__Consolidado EE', 'Vapor L3__Consolidado GasVapor', 'Totalizador_Aire_L3__Consolidado Aire', 'Totalizador_Aire_L5__Consolidado Aire', 'dow_sin', 'dow_cos', 'mes_sin', 'mes_cos', 'fin_de_semana', 'KW Gral Planta__Consolidado EE_x_dow_sin', 'Planta (Kw)__Consolidado EE_x_dow_sin', 'Servicios (Kw)__Consolidado EE_x_dow_cos', 'KW Gral Planta__Consolidado EE_x_dow_cos', 'Planta (Kw)__Consolidado EE_x_dow_cos', 'Agua Planta (Hl)__Consolidado Agua_x_dow_cos', 'Servicios (Kw)__Consolidado EE_x_mes_sin', 'KW Gral Planta__Consolidado EE_x_mes_sin', 'Planta (Kw)__Consolidado EE_x_mes_sin', 'Agua Planta (Hl)__Consolidado Agua_x_mes_sin', 'KW Gral Planta__Consolidado EE_x_mes_cos', 'Planta (Kw)__Consolidado EE_x_mes_cos', 'Sala Maq (Kw)__Consolidado EE_x_finde', 'Servicios (Kw)__Consolidado EE_x_finde', 'KW Gral Planta__Consolidado EE_x_finde', 'Planta (Kw)__Consolidado EE_x_finde', 'Agua Planta (Hl)__C

#### Método de Lasso y ElasticNet

In [11]:
# --- D. Método Embebido (LassoCV) ---
scaler_reg = StandardScaler()

scaler_reg.fit(X_train)

X_train_scaled_reg = pd.DataFrame(scaler_reg.transform(X_train.fillna(0)), columns=X_train.columns)
X_test_scaled_reg = pd.DataFrame(scaler_reg.transform(X_test.fillna(0)), columns=X_test.columns)

lasso = LassoCV(cv=5, random_state=42).fit(X_train_scaled_reg, y_train)

coef_lasso = pd.Series(lasso.coef_, index=X_train.columns)
features_lasso = coef_lasso.abs().sort_values(ascending=False).head(30).index.tolist()
print(f"\n--- 4. Método Lasso (top 30) --- \n{features_lasso}")

# --- E. Método Embebido (ElasticNetCV) ---
elastic = ElasticNetCV(cv=5, random_state=42).fit(X_train_scaled_reg, y_train)

coef_elastic = pd.Series(elastic.coef_, index=X_train.columns)
features_elastic = coef_elastic.abs().sort_values(ascending=False).head(30).index.tolist()
print(f"\n--- 5. Método ElasticNet (top 30) --- \n{features_elastic}")

c:\Users\out6let\miniconda3\envs\cervecera_env\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:681: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.957e+06, tolerance: 4.830e+06
  model = cd_fast.enet_coordinate_descent_gram(
c:\Users\out6let\miniconda3\envs\cervecera_env\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:681: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 6.414e+06, tolerance: 4.830e+06
  model = cd_fast.enet_coordinate_descent_gram(
c:\Users\out6let\miniconda3\envs\cervecera_env\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:681: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the feat


--- 4. Método Lasso (top 30) --- 
['Frio', 'Frio_ma7', 'mes_cos', 'ratio_Bodega_sobre_Sala_Maq', 'ratio_Servicios_sobre_Bodega', 'mes_sin', 'estacion_cos', 'Red Paste L4__Consolidado Agua', 'ET Servicios / Hl__Consolidado KPI', 'Sala Maq (Kw)__Consolidado EE_x_dow_cos', 'Aire Envasado (M3)__Consolidado Aire', 'Agua Bodega / Hl__Consolidado KPI', 'KW Gral Planta__Consolidado EE', 'Aire Bodega / Hl__Consolidado KPI', 'Sala Maq (Kw)__Consolidado EE', 'Frio_lag7', 'Servicios (Kw)__Consolidado EE', 'Agua Planta (Hl)__Consolidado Agua_x_dow_sin', 'Planta (Kw)__Consolidado EExAgua Planta (Hl)__Consolidado Agua', 'Servicios (Kw)__Consolidado EExPlanta (Kw)__Consolidado EE', 'Servicios (Kw)__Consolidado EExAgua Planta (Hl)__Consolidado Agua', 'KW Gral Planta__Consolidado EE_x_dow_cos', 'KW Gral Planta__Consolidado EExPlanta (Kw)__Consolidado EE', 'KW Gral Planta__Consolidado EExAgua Planta (Hl)__Consolidado Agua', 'Sala Maq (Kw)__Consolidado EE_x_dow_sin', 'Servicios (Kw)__Consolidado EE_x_dow

#### SequencialFeatureSelector

In [12]:
# # --- F. Método Wrapper (SequentialFeatureSelector) ---
# # (Usamos los datos escalados con StandardScaler, que ya están limpios de NaNs)
# print("\n--- 6. Método Wrapper (SFS) ---")
# print("Iniciando SFS... (Esto puede tardar varios minutos)...")

# # 1. Definir el modelo que usará SFS para evaluar
# # (Tu código usaba GradientBoosting, es una buena elección)
# modelo_sfs = GradientBoostingRegressor(random_state=42)

# # 2. APRENDER (fit) el selector SFS SOLO en X_train
# sfs = SequentialFeatureSelector(modelo_sfs, 
#                               n_features_to_select=30, # ¡Aquí pones 30!
#                               direction='forward', # 'forward' es más rápido que 'backward'
#                               cv=3, # 3 splits es más rápido que 5
#                               n_jobs=-1) # Usar todos los cores

# sfs.fit(X_train_scaled_reg, y_train)

# # 3. OBTENER las 30 mejores features
# features_sfs = X_train.columns[sfs.get_support()]
# print(f"SFS completado.\nFeatures seleccionadas: \n{features_sfs.tolist()}")

--- 6. Método Wrapper (SFS) ---
Features seleccionadas por SFS: 


['Agua Elab / Hl__Consolidado KPI', 'Agua Planta de Agua/Hl__Consolidado KPI', 'Produccion Agua / Hl__Consolidado KPI', 'EE Envasado / Hl__Consolidado KPI', 'Agua Bodega / Hl__Consolidado KPI', 'EE Elaboracion / Hl__Consolidado KPI', 'EE Linea 4 / Hl__Consolidado KPI', 'Aire Elaboracion / Hl__Consolidado KPI', 'KW Enfluentes Hidr__Totalizadores Energia', 'KW Obrador Contratistas__Totalizadores Energia', 'KW Mycom 3__Totalizadores Energia', 'KW Mycom 7__Totalizadores Energia', 'FC Lavadora L3__Consolidado Agua', 'Produccion (Hl)__Consolidado Agua', 'Frio', 'Frio_lag7', 'Frio_ma7', 'dow_sin', 'dow_cos', 'mes_sin', 'fin_de_semana', 'Servicios (Kw)__Consolidado EE_x_dow_sin', 'KW Gral Planta__Consolidado EE_x_dow_sin', 'KW Gral Planta__Totalizadores Energia_x_dow_sin', 'Planta (Kw)__Consolidado EE_x_dow_sin', 'KW Gral Planta__Consolidado EE_x_dow_cos', 'KW Gral Planta__Totalizadores Energia_x_dow_cos', 'Servicios (Kw)__Consolidado EE_x_finde', 'Bodega_sum', 'estacion_sin']

#### Random Forest

In [13]:
rf = RandomForestRegressor(n_estimators=300, random_state=42, n_jobs=-1)
rf.fit(X_train, y_train)

perm_train = permutation_importance(
    rf, X_train, y_train,
    n_repeats=5, random_state=42, n_jobs=-1,
    scoring="neg_mean_absolute_error"
)

In [14]:
perm_imp_train = pd.Series(perm_train.importances_mean, index=X_train.columns).sort_values(ascending=False)

features_rf = perm_imp_train.head(30).index.tolist()

X_train_sel = X_train[features_rf].copy()
X_test_sel  = X_test[features_rf].copy()

rf_sel = RandomForestRegressor(n_estimators=300, random_state=42, n_jobs=-1)
rf_sel.fit(X_train_sel, y_train)

y_pred = rf_sel.predict(X_test_sel)

#### Comparación de métodos

In [15]:
model = RandomForestRegressor(n_estimators=300, random_state=42, n_jobs=-1)

# Un diccionario con todas tus listas de 30 features (de los pasos anteriores)
listas_de_features = {
    "Varianza": features_variance,
    "F-Score": features_fscore,
    "Mutual_Info": features_mi,
    "Lasso": features_lasso,
    "ElasticNet": features_elastic,
    #"Wrapper_SFS": features_sfs,
    "Random Forest": features_rf
}

def evaluar_en_test(X_train_subset, y_train, X_test_subset, y_test, nombre_metodo):
    # --- A. Escalar (SIN DATA LEAKAGE) ---
    # Limpiamos NaNs que puedan quedar
    X_train_clean = X_train_subset.fillna(0)
    X_test_clean = X_test_subset.fillna(0)
    
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train_clean)
    
    X_test_scaled = scaler.transform(X_test_clean) 
    
    
    model.fit(X_train_scaled, y_train)
    
    y_pred = model.predict(X_test_scaled)
    
    r2 = r2_score(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_test, y_pred) 
    
    # 5. Imprimir (como en tu código)
    print(f"\n>>> {nombre_metodo} ({X_train_subset.shape[1]} features) <<<")
    print(f"  R2 (en Test):   {r2:.4f}")
    print(f"  RMSE (en Test):  {rmse:.2f}")
    print(f"  MAE (en Test):  {mae:.2f}") 

    
for nombre, lista_de_features in listas_de_features.items():
    X_train_subset = X_train[lista_de_features]
    X_test_subset = X_test[lista_de_features]
    
    evaluar_en_test(X_train_subset, y_train, X_test_subset, y_test, nombre)


>>> Varianza (30 features) <<<
  R2 (en Test):   0.1733
  RMSE (en Test):  5030.63
  MAE (en Test):  4075.14

>>> F-Score (30 features) <<<
  R2 (en Test):   0.5975
  RMSE (en Test):  3510.04
  MAE (en Test):  2665.89

>>> Mutual_Info (30 features) <<<
  R2 (en Test):   0.5703
  RMSE (en Test):  3626.70
  MAE (en Test):  2797.06

>>> Lasso (30 features) <<<
  R2 (en Test):   0.6009
  RMSE (en Test):  3495.27
  MAE (en Test):  2654.54

>>> ElasticNet (30 features) <<<
  R2 (en Test):   0.5920
  RMSE (en Test):  3533.84
  MAE (en Test):  2711.06

>>> Random Forest (30 features) <<<
  R2 (en Test):   0.6268
  RMSE (en Test):  3380.04
  MAE (en Test):  2565.61


#### Elección de variables y creación de los df

In [16]:
cols_ok = [c for c in features_rf if c in df_train_final.columns]

# Avisar si alguna feature no aparece en df_train
faltan = sorted(set(features_rf) - set(cols_ok))
if faltan:
    print("No encontré estas columnas en df_train:", faltan)

df_train = df_train_final.loc[:, cols_ok + ['y'] + ['dia']].copy()
df_test = df_test_final.loc[:, cols_ok + ['y'] + ['dia']].copy()

In [17]:
cols_ok

['Frio_ma7',
 'Frio',
 'ratio_Bodega_sobre_Sala_Maq',
 'ratio_Sala_Maq_sobre_Bodega',
 'Sala Maq (Kw)__Consolidado EExServicios (Kw)__Consolidado EE',
 'Sala Maq (Kw)__Consolidado EE',
 'Bodega_share',
 'Sala Maq (Kw)__Consolidado EE_x_mes_cos',
 'ratio_Servicios_sobre_Bodega',
 'ratio_Bodega_sobre_Servicios',
 'Servicios (Kw)__Consolidado EE_x_mes_cos',
 'Sala_Maq_sum',
 'Sala Maq (Kw)__Consolidado EExPlanta (Kw)__Consolidado EE',
 'Frio_lag7',
 'Frio_ma3',
 'Agua Planta (Hl)__Consolidado Agua_x_mes_cos',
 'Sala Maq (Kw)__Consolidado EExKW Gral Planta__Consolidado EE',
 'Sala Maq (Kw)__Consolidado EE_x_mes_sin',
 'Aire L4 / Hl__Consolidado KPI',
 'KW Gral Planta__Consolidado EE_x_mes_cos',
 'Linea 3 (Kw)__Consolidado EE',
 'Servicios (Kw)__Consolidado EE',
 'Red L1 y L2__Consolidado Agua',
 'Sala Maq (Kw)__Consolidado EExAgua Planta (Hl)__Consolidado Agua',
 'Agua Linea 3/Hl__Consolidado KPI',
 'Sala Maq (Kw)__Consolidado EE_x_dow_cos',
 'CO 2 Linea 4 / Hl__Consolidado KPI',
 'Red Pas

['Frio_ma7',
 'Frio',
 'ratio_Bodega_sobre_Sala_Maq',
 'ratio_Sala_Maq_sobre_Bodega',
 'Sala Maq (Kw)__Consolidado EExServicios (Kw)__Consolidado EE',
 'Sala Maq (Kw)__Consolidado EE',
 'Bodega_share',
 'Sala Maq (Kw)__Consolidado EE_x_mes_cos',
 'ratio_Servicios_sobre_Bodega',
 'Servicios (Kw)__Consolidado EE_x_mes_cos',
 'ratio_Bodega_sobre_Servicios',
 'Sala_Maq_sum',
 'Sala Maq (Kw)__Consolidado EExPlanta (Kw)__Consolidado EE',
 'Frio_lag7',
 'Frio_ma3',
 'Agua Planta (Hl)__Consolidado Agua_x_mes_cos',
 'Sala Maq (Kw)__Consolidado EE_x_mes_sin',
 'Sala Maq (Kw)__Consolidado EExKW Gral Planta__Consolidado EE',
 'Aire L4 / Hl__Consolidado KPI',
 'KW Gral Planta__Consolidado EE_x_mes_cos',
 'Linea 3 (Kw)__Consolidado EE',
 'Servicios (Kw)__Consolidado EE',
 'Red L1 y L2__Consolidado Agua',
 'Sala Maq (Kw)__Consolidado EExAgua Planta (Hl)__Consolidado Agua',
 'Agua Linea 3/Hl__Consolidado KPI',
 'Sala Maq (Kw)__Consolidado EE_x_dow_cos',
 'CO 2 Linea 4 / Hl__Consolidado KPI',
 'Red Paste L4__Consolidado Agua',
 'ET Linea 3/Hl__Consolidado KPI',
 'Totalizador_Aire_Cocina__Consolidado Aire']

### Pipeline para escalado y transformacióon

In [18]:
def boxcox_transform(X):
    X = np.where(X <= 0, X + 1e-9, X)
    return PowerTransformer(method='box-cox', standardize=True).fit_transform(X)

def log_transform(X):
    return np.log1p(np.clip(X, a_min=0, a_max=None))

def sqrt_transform(X):
    return np.sqrt(np.abs(X))


ycol = 'y'
non_features = [ycol, 'dia'] 

# 1a. Crear X_train, y_train (para ENTRENAR el GridSearchCV)
X_train_full = df_train.drop(columns=non_features, errors='ignore')
y_train = df_train[ycol]

# 1b. Crear X_test, y_test (para EVALUAR AL FINAL)
X_test_full = df_test.drop(columns=non_features, errors='ignore')
y_test = df_test[ycol]

# 1c. Detectar features numéricas (APRENDIENDO SOLO DE X_TRAIN)
numeric_features = X_train_full.select_dtypes(include=np.number).columns.tolist()

# 1d. Asegurarnos que X_train y X_test tengan las mismas columnas numéricas
X_train = X_train_full[numeric_features].fillna(0)
X_test = X_test_full[numeric_features].fillna(0) # Rellenamos NaNs simple

# --- 2. DEFINICIÓN DEL PIPELINE (Tu código, sin cambios) ---
# (El imputer SimpleImputer(median) es bueno para velocidad en el gridsearch)
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', 'passthrough'),
    ('power', 'passthrough')
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features)
    ],
    remainder='passthrough'
)

model_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', Ridge()) # Ponemos un modelo default, se va a reemplazar
])

# --- 3. ESPACIO DE BÚSQUEDA (Tu código, sin cambios) ---
param_grid = {
    'preprocessor__num__scaler': [ 
        'passthrough',
        StandardScaler(),
        MinMaxScaler(),
        RobustScaler()
    ],
    
    
    'preprocessor__num__power': [ 
        PowerTransformer(method='yeo-johnson'),
        FunctionTransformer(boxcox_transform, validate=False),
        FunctionTransformer(log_transform, validate=False),
        FunctionTransformer(sqrt_transform, validate=False),
        'passthrough'
    ],
    

    'regressor': [
        XGBRegressor(
            n_estimators=600,
            learning_rate=0.05,
            max_depth=6,
            subsample=0.8,
            colsample_bytree=0.8,
            reg_alpha=0.0,
            reg_lambda=1.0,
            random_state=42,
            n_jobs=-1,
            tree_method="hist"
        ),
        RandomForestRegressor(
            n_estimators=600,
            max_depth=None,
            min_samples_split=2,
            min_samples_leaf=1,
            max_features="sqrt",
            bootstrap=True,
            n_jobs=-1,
            random_state=42
        ),
        LGBMRegressor(
            n_estimators=1000,
            learning_rate=0.05,
            num_leaves=31,
            max_depth=-1,
            subsample=0.8,
            colsample_bytree=0.8,
            reg_lambda=1.0,
            random_state=42,
            n_jobs=-1
        ),
        Ridge(
            alpha=1.0,
            fit_intercept=True
        ),
        Lasso(
            alpha=0.001,
            max_iter=10000,
            fit_intercept=True
        )
    ],
}

In [ ]:
cv = KFold(n_splits=5, shuffle=True, random_state=42) # Tu CV original

grid_search = GridSearchCV(
    model_pipeline, 
    param_grid, 
    cv=cv, # Reemplaza por tscv si quieres probar
    scoring='neg_mean_absolute_error', # O 'neg_mean_absolute_error' para el MAE
    n_jobs=-1, 
    verbose=1
)

grid_search.fit(X_train, y_train)

# Imprimir los mejores parámetros (encontrados en Train)
print("\nLos mejores parámetros encontrados (en train) son:", grid_search.best_params_)
print("El mejor MAE (en CV de train) es: {:.4f}".format(grid_search.best_score_))

best_pipeline = grid_search.best_estimator_

y_pred = best_pipeline.predict(X_test)

r2_test = r2_score(y_test, y_pred)
mae_test = mean_absolute_error(y_test, y_pred) # La métrica que te pide el TP
mse_test  = mean_squared_error(y_test, y_pred)
rmse_test = np.sqrt(mse_test)

print(f"R² (en Test):  {r2_test:.4f}")
print(f"MAE (en Test): {mae_test:.2f}")
print(f"RMSE (en Test): {rmse_test:.2f}")

Fitting 5 folds for each of 100 candidates, totalling 500 fits


c:\Users\Usuario\miniconda3\envs\cervecera_env\Lib\site-packages\sklearn\model_selection\_validation.py:516: FitFailedWarning: 
75 fits failed out of a total of 500.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
75 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\Usuario\miniconda3\envs\cervecera_env\Lib\site-packages\sklearn\model_selection\_validation.py", line 859, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\Usuario\miniconda3\envs\cervecera_env\Lib\site-packages\sklearn\base.py", line 1365, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Usuario\miniconda3\envs\cervecera_e


Los mejores parámetros encontrados (en train) son: {'preprocessor__num__power': PowerTransformer(), 'preprocessor__num__scaler': MinMaxScaler(), 'regressor': Ridge()}
El mejor MAE (en CV de train) es: -2841.3129
R² (en Test):  0.5968
MAE (en Test): 2744.29
RMSE (en Test): 52.39


### Cargar datos escalados y transformados

In [27]:
# --- 0. Imports (Asegúrate de tenerlos) ---
import pandas as pd
import numpy as np
import json
import datetime
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler, PowerTransformer
from pandas.api.types import is_numeric_dtype # Para chequear tipos

# --- (ASUMIMOS QUE YA CARGASTE ESTOS 2 OBJETOS) ---
# df_train: El dataframe COMPLETO (train+val) con 'dia', 'y', y features numéricas
# df_test: El dataframe COMPLETO (test) con 'dia', 'y', y features numéricas
# (Ignoramos las variables X_train y X_test viejas, estaban sucias)
# -----------------------------------------------------------------

# --- 1. DEFINIR CORTE Y SEPARAR DATAFRAMES (Tu código, está perfecto) ---
CUTOFF_DATE = '2022-07-01'
print(f"Separando Train / Val en la fecha: {CUTOFF_DATE}")


# Separamos el df_train original en "solo train" y "solo val"
df_train_noval = df_train[df_train['dia'] <= CUTOFF_DATE].copy()
df_val = df_train[df_train['dia'] > CUTOFF_DATE].copy()
df_test_final = df_test.copy() # df_test ya estaba separado

print(f"Forma df_train_noval: {df_train_noval.shape}")
print(f"Forma df_val: {df_val.shape}")
print(f"Forma df_test_final: {df_test_final.shape}")


# --- 2. OBTENER LOS SETS NUMÉRICOS (X) SIN LEAKAGE (LA CORRECCIÓN) ---

# ¡IMPORTANTE! Define aquí tu columna objetivo y otras que NO se escalan
# (Ej. 'Frio' es el target crudo, 'y' es el target transformado)
TARGET = 'y'
COLS_NO_NUMERICAS = ['dia', TARGET] # ¡AJUSTA ESTO SI ES NECESARIO!

# Identificamos las columnas numéricas (features) dinámicamente
# "col es numérica" Y "col no está en la lista de excluidas"
numeric_cols = [
    col for col in df_train_noval.columns 
    if is_numeric_dtype(df_train_noval[col]) and col not in COLS_NO_NUMERICAS
]
print(f"\nSe identificaron {len(numeric_cols)} columnas numéricas para escalar.")

# Ahora creamos los 3 sets numéricos (X) a partir de los dataframes separados
# Esta vez, SÍ va a funcionar porque numeric_cols viene de df_train_noval
X_train_noval_numeric = df_train_noval[numeric_cols]
X_val_numeric = df_val[numeric_cols]
X_test_numeric = df_test_final[numeric_cols]

print(f"Forma X_train_noval_numeric: {X_train_noval_numeric.shape}")
print(f"Forma X_val_numeric: {X_val_numeric.shape}")
print(f"Forma X_test_numeric: {X_test_numeric.shape}")


# --- 3. Definir el pipeline de preprocesamiento (Tu código) ---
preproc_pipeline = Pipeline(steps=[
    ('scaler', MinMaxScaler()),
    ('power', PowerTransformer(method='yeo-johnson')) 
])


# --- 4. APLICAR (FIT Y TRANSFORM) SIN LEAKAGE (Tu código, ahora funciona) ---
print("\nAjustando (fit) el pipeline SÓLO en X_train_noval_numeric...")

# ¡AQUÍ ESTÁ LA MAGIA! Fiteamos SOLO con "solo train"
preproc_pipeline.fit(X_train_noval_numeric)

# Ahora transformamos los 3 sets con el mismo pipeline (ya fiteado)
print("Transformando X_train_noval, X_val, y X_test...")
X_train_noval_scaled_array = preproc_pipeline.transform(X_train_noval_numeric)
X_val_scaled_array = preproc_pipeline.transform(X_val_numeric)
X_test_scaled_array = preproc_pipeline.transform(X_test_numeric)


# --- 5. CONSTRUIR LOS 3 DATAFRAMES FINALES ---
# (Esta parte estaba bien, usará los nuevos arrays)

# --- df_train_final ---
df_train_final = df_train_noval.copy()
df_train_scaled_features = pd.DataFrame(
    X_train_noval_scaled_array, 
    columns=numeric_cols, 
    index=df_train_noval.index
)
df_train_final.update(df_train_scaled_features)
df_train_final = df_train_final.drop(columns=['dia'])

# --- df_val_final ---
df_val_final = df_val.copy()
df_val_scaled_features = pd.DataFrame(
    X_val_scaled_array, 
    columns=numeric_cols, 
    index=df_val.index
)
df_val_final.update(df_val_scaled_features)
df_val_final = df_val_final.drop(columns=['dia'])

# --- df_test_final ---
# (df_test_final ya existe, solo actualizamos las features)
df_test_scaled_features = pd.DataFrame(
    X_test_scaled_array, 
    columns=numeric_cols, 
    index=df_test_final.index
)
df_test_final.update(df_test_scaled_features)
df_test_final = df_test_final.drop(columns=['dia'])

# --- 6. Guardar en CSV (3 archivos) ---
ruta_train_csv = '../data/dataset_train_final.csv'
ruta_val_csv = '../data/dataset_val_final.csv'
ruta_test_csv = '../data/dataset_test_final.csv'

df_train_final.to_csv(ruta_train_csv, index=False)
df_val_final.to_csv(ruta_val_csv, index=False)
df_test_final.to_csv(ruta_test_csv, index=False)

# --- 7. Guardar Linaje (Actualizado) ---
info_linaje = {
    'descripcion': 'Datasets finales procesados, escalados SIN LEAKAGE, y separados en train/val/test.',
    'fuentes_creadas': [ruta_train_csv, ruta_val_csv, ruta_test_csv],
    'script_transformacion': 'notebooks/preprocesamiento.ipynb', 
    'fecha_creacion': datetime.datetime.now().isoformat(),
    'cutoff_date_train_val': CUTOFF_DATE
}

ruta_json = '../data/data_lineage.json'
with open(ruta_json, 'w') as f:
    json.dump(info_linaje, f, indent=4)

print(f"\n¡Linaje de datos guardado en {ruta_json}!")

print(f"\n¡Listo! Archivos COMPLETOS y SIN LEAKAGE guardados:")
print(f"Train escalado: {ruta_train_csv} (Shape: {df_train_final.shape})")
print(f"Val escalado:   {ruta_val_csv} (Shape: {df_val_final.shape})")
print(f"Test escalado:  {ruta_test_csv} (Shape: {df_test_final.shape})")

Separando Train / Val en la fecha: 2022-07-01
Forma df_train_noval: (724, 32)
Forma df_val: (182, 32)
Forma df_test_final: (297, 32)

Se identificaron 30 columnas numéricas para escalar.
Forma X_train_noval_numeric: (724, 30)
Forma X_val_numeric: (182, 30)
Forma X_test_numeric: (297, 30)

Ajustando (fit) el pipeline SÓLO en X_train_noval_numeric...
Transformando X_train_noval, X_val, y X_test...

¡Linaje de datos guardado en ../data/data_lineage.json!

¡Listo! Archivos COMPLETOS y SIN LEAKAGE guardados:
Train escalado: ../data/dataset_train_final.csv (Shape: (724, 31))
Val escalado:   ../data/dataset_val_final.csv (Shape: (182, 31))
Test escalado:  ../data/dataset_test_final.csv (Shape: (297, 31))


### Chechksum

In [29]:
import sys

src_path = os.path.abspath(os.path.join(os.getcwd(), '..', 'src'))

# 2. Agrega 'src' al path de Python (si no está ya)
if src_path not in sys.path:
    sys.path.append(src_path)
    print(f"Carpeta '{src_path}' agregada al path.")

from tools import checksum

Carpeta 'c:\Users\out6let\Desktop\REPOSITORIO\TPF\src' agregada al path.


In [30]:
df_train.shape

(906, 32)

In [31]:
checksum(df_train_final, "df_train")  
checksum(df_test_final, "df_test")
checksum(df_val_final, "df_val")

'Checksum calculado y guardado: 9e0c562bf4ba40a3479bc90ce0858817'